In [ ]:
# Import packages

# Dataframe Packages
import numpy as np
import xarray as xr
import pandas as pd

# Vector Packages
#import geopandas as gpd
import shapely

# Raster Packages
import rioxarray as rxr
from rioxarray.merge import merge_arrays
import rasterstats as rs

# Data Access Packages
import earthaccess as ea
#from nsidc_fetch import download, format_date, format_boundingbox
import h5py
import pickle
from tensorflow.keras.models import load_model

# General Packages
import os
import re
import shutil
from datetime import datetime
import glob
from pprint import pprint
from typing import Union
from pathlib import Path
from tqdm import tqdm
import time
import requests

In [5]:
import NSIDC_Data

short_name = 'ASO_50M_SWE'
version = '1'
polygon = ''
filename_filter = ''
url_list = []

CMR_URL = 'https://cmr.earthdata.nasa.gov'
URS_URL = 'https://urs.earthdata.nasa.gov'
CMR_PAGE_SIZE = 2000
CMR_FILE_URL = ('{0}/search/granules.json?provider=NSIDC_ECS'
                '&sort_key[]=start_date&sort_key[]=producer_granule_id'
                '&scroll=true&page_size={1}'.format(CMR_URL, CMR_PAGE_SIZE))

# Sample list of regions with indices
Region_list =  ['N_Sierras',
                'S_Sierras',
                'Greater_Yellowstone',
                'N_Co_Rockies',
                'SW_Mont',
                'SW_Co_Rockies',
                'GBasin',
                'N_Wasatch',
                'N_Cascade',
                'S_Wasatch',
                'SW_Mtns',
                'E_WA_N_Id_W_Mont',
                'S_Wyoming',
                'SE_Co_Rockies',
                'Sawtooth',
                'Ca_Coast',
                'E_Or',
                'N_Yellowstone',
                'S_Cascade',
                'Wa_Coast',
                'Greater_Glacier',
                'Or_Coast']

def get_bounding_box(region):
  
    regions = pd.read_pickle("C:\\Users\\VISH NU\\National_snow_model\\National-Snow-Model\\Data\\Processed\\RegionVal.pkl")
    superset = []
    
    superset.append(regions[region])
    superset = pd.concat(superset)
    superset = gpd.GeoDataFrame(superset, geometry=gpd.points_from_xy(superset.Long, superset.Lat, crs="EPSG:4326"))
    bounding_box = list(superset.total_bounds)
    
    return f"{bounding_box[0]},{bounding_box[1]},{bounding_box[2]},{bounding_box[3]}"


def main(argv = None):
    global short_name, version, time_start, time_end, bounding_box, \
        polygon, filename_filter, url_list
    
    start = 'T00:00:00Z'
    end = 'T23:59:59Z'
    time_start = input("Enter start date (YYYY-MM-DD): ") #2013-04-02
    time_start = time_start + start
 
    time_end = input("Enter end date (YYYY-MM-DD): ") #2019-07-19
    time_end = time_end + end
    current_directory = os.getcwd()
    folder = "SWE_Data"
    directory = os.path.join(current_directory, folder)
    
    print("Select a region by entering its index:")
    for i, region in enumerate(Region_list, start=1):
        print(f"{i}. {region}")

    try:
        user_input = int(input("Enter the index of the region: "))
        if 1 <= user_input <= len(Region_list):
            selected_region = Region_list[user_input - 1]
            bounding_box = get_bounding_box(selected_region)
            
            print(f"You selected: {selected_region}")
            print(f"Bounding Box: {bounding_box}")
        else:
            print("Invalid index. Please select a valid index.")
    except ValueError:
        print("Invalid input. Please enter a valid index.")
    print(f"Fetching file URLs in progress for {selected_region} from {time_start} to {time_end}")
    try:
        if not url_list:
            url_list = NSIDC_Data.cmr_search(short_name, version, time_start, time_end,
                                  bounding_box=bounding_box, polygon=polygon,
                                  filename_filter=filename_filter, quiet=False)
        
        NSIDC_Data.cmr_download(url_list, directory, False)
    except KeyboardInterrupt:
        quit()

if __name__ == "__main__":
    main()

Enter start date (YYYY-MM-DD): 2013-04-02
Enter end date (YYYY-MM-DD): 2019-07-19
Select a region by entering its index:
1. N_Sierras
2. S_Sierras
3. Greater_Yellowstone
4. N_Co_Rockies
5. SW_Mont
6. SW_Co_Rockies
7. GBasin
8. N_Wasatch
9. N_Cascade
10. S_Wasatch
11. SW_Mtns
12. E_WA_N_Id_W_Mont
13. S_Wyoming
14. SE_Co_Rockies
15. Sawtooth
16. Ca_Coast
17. E_Or
18. N_Yellowstone
19. S_Cascade
20. Wa_Coast
21. Greater_Glacier
22. Or_Coast
Enter the index of the region: 2
You selected: S_Sierras
Bounding Box: -120.3763448720203,36.29256774541929,-118.292253412863,38.994985247736324
Fetching file URLs in progress for S_Sierras from 2013-04-02T00:00:00Z to 2019-07-19T23:59:59Z
Querying for data:
	https://cmr.earthdata.nasa.gov/search/granules.json?provider=NSIDC_ECS&sort_key[]=start_date&sort_key[]=producer_granule_id&scroll=true&page_size=2000&short_name=ASO_50M_SWE&version=001&version=01&version=1&temporal[]=2013-04-02T00:00:00Z,2019-07-19T23:59:59Z&bounding_box=-120.3763448720203,36.292

Earthdata Login Username: vishnugindi
Earthdata Login Password: ········
dmlzaG51Z2luZGk6UHJvY2Vzc2luZzEh
001/188: C:\Users\VISH NU\NSM_SSWEET\SWE_Data\ASO_50M_SWE_USCATB_20130403.tif
  [============                                                ]  20%  3.4MB/s    [========================                                    ]  40%  5.8MB/s    [====================================                        ]  60%  7.5MB/s    [================================================            ]  80%  5.7MB/s    [============================================================] 100%  6.7MB/s   
002/188: C:\Users\VISH NU\NSM_SSWEET\SWE_Data\ASO_50M_SWE_USCATB_20130403.tif.xml


In [3]:
import pandas

rds = rxr.open_rasterio(
    r"C:\Users\VISH NU\NSM_SSWEET\SWE_Data\ASO_50M_SWE_USCACE_20160401.tif")
rds = rds.squeeze().drop("spatial_ref").drop("band")
rds.name = "data"
df = rds.to_dataframe().reset_index()
df[df.data>=0.0].to_csv("out.csv", index=False)

In [4]:
df

,y,x,data
0,4.234823e+06,242740.000051,-9999.0
1,4.234823e+06,242790.000152,-9999.0
2,4.234823e+06,242840.000253,-9999.0
3,4.234823e+06,242890.000354,-9999.0
4,4.234823e+06,242940.000455,-9999.0
...,...,...,...
1451338,4.178923e+06,307340.130658,-9999.0
1451339,4.178923e+06,307390.130759,-9999.0
1451340,4.178923e+06,307440.130860,-9999.0
1451341,4.178923e+06,307490.130961,-9999.0


In [ ]:
##Sample Code##
Reprj_ds = gdal.Warp("output_tiff.tif", ds, dstSRS = "EPSG:4326", xRes = 0.01, yRes = -0.01, 
                            resampleAlg = "bilinear")
print(Reprj_ds.GetGeoTransform())
print(Reprj_ds.GetProjection())
rds = rxr.open_rasterio(
    r"C:\Users\VISH NU\National_snow_model\National-Snow-Model\Model\output_tiff.tif")
rds = rds.squeeze().drop("spatial_ref").drop("band")
rds.name = "data"
df = rds.to_dataframe().reset_index()
df

In [ ]:
from shapely.geometry import Point

def processing_tiff(input_file, output_res):
    try:
        # Extract date from the input file name
        date = os.path.splitext(input_file)[0].split("_")[-1]
        
        # Define the output file path
        output_folder = os.path.join(os.getcwd(), "Processed_data")
        os.makedirs(output_folder, exist_ok=True)
        output_file = os.path.join(output_folder, f"ASO_100M_{date}.tif")

        # Open the input TIFF file
        ds = gdal.Open(input_file)

        if ds is None:
            print(f"Failed to open '{input_file}'. Make sure the file is a valid GeoTIFF file.")
            return None
        
        # Reproject and resample
        gdal.Warp(output_file, ds, dstSRS="EPSG:4326", xRes=output_res, yRes=-output_res, resampleAlg="bilinear")

        # Read the processed TIFF file using rasterio
        rds = rxr.open_rasterio(output_file)
        rds = rds.squeeze().drop("spatial_ref").drop("band")
        rds.name = "data"
        df = rds.to_dataframe().reset_index()
        return df

    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None
    

def convert_tiff_to_csv(folder_name, cwd, output_res):
    # Check if the folder exists
    folder_path = os.path.join(cwd, folder_name)
    if not os.path.exists(folder_path) or not os.path.isdir(folder_path):
        print(f"The folder '{folder_name}' does not exist.")
        return
    # Check if the folder is empty
    if not os.listdir(folder_path):
        print(f"The folder '{folder_name}' is empty.")
        return

    # Get a list of TIFF files in the folder
    tiff_files = [filename for filename in os.listdir(folder_path) if filename.endswith(".tif")]

    # Create CSV files from TIFF files
    for tiff_filename in tiff_files:
        # Open the TIFF file
        tiff_filepath = os.path.join(folder_path, tiff_filename)
        df = processing_tiff(tiff_filepath, output_res)

        if df is not None:
            # Get the date from the TIFF filename
            date = os.path.splitext(tiff_filename)[0].split("_")[-1]

            # Define the CSV filename and folder
            csv_filename = f"ASO_SWE_{date}.csv"
            csv_folder = os.path.join(cwd, "Processed_Data", "SWE_csv")
            os.makedirs(csv_folder, exist_ok=True)
            csv_filepath = os.path.join(csv_folder, csv_filename)

            # Save the DataFrame as a CSV file
            df.to_csv(csv_filepath, index=False)

            print(f"Converted '{tiff_filename}' to '{csv_filename}'")
            

def processing_CSVs(folder_name, cwd):
    gridcells_df = gpd.read_file(
        r"C:\Users\VISH NU\NSM_EvaluationTool\Standardized-Snow-Water-Equivalent-Evaluation-Tool\SSWEET\Provided_Data\grid_cells.geojson")
    gridcells_df = gridcells_df[gridcells_df['region'] == 'sierras']
    
    folder_path = os.path.join(cwd, folder_name)
    CSV_files = [filename for filename in os.listdir(folder_path) if filename.endswith(".csv")]
    
    for csv_filename in CSV_files:
        
        SWE_ASO = pd.read_csv(r"C:\Users\VISH NU\NSM_SSWEET\Processed_Data\SWE_csv\ASO_SWE_20130403.tif.csv")
    
        geo = [Point(xy) for xy in zip(SWE_ASO['x'], SWE_ASO['y'])]
        SWE_gdf = gpd.GeoDataFrame(SWE_ASO, geometry=geo)

        SWE_gdf.crs = gridcells_df.crs
        joined_df = gpd.sjoin(SWE_gdf, gridcells_df, how="inner", op="within")
    
        Final_df = joined_df.drop(['geometry', 'index_right'], axis=1)
        Final_df.rename(columns={'data': 'swe'}, inplace=True)  
        Final_df.to_csv(***csv files with cell_ids***, index=False)

        return 'Converted'

In [ ]:
cwd = "C:\\Users\\VISH NU\\NSM_SSWEET"
folder_name = "SWE_Data"
output_resolution = 0.01  # Replace with your desired resolution
convert_tiff_to_csv(folder_name, cwd, output_resolution)